In [4]:
# get to working directory of llms
import os
os.chdir('/pub/anvieyra/copilot/llms') 
from user_modules.db import Session, get_data
from user_modules.table_instances import Dialogues
from user_modules.helper_functions import generate_sentiment 
import pandas as pd
import re
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch
import transformers

# import key performance stats
import time
import GPUtil
import logging

Get data from server

In [2]:
dialogue_data = get_data('training')
dialogue_data.head()

2024-05-05 16:26:54,713 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-05-05 16:26:54,714 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-05 16:26:54,834 INFO sqlalchemy.engine.Engine select current_schema()
2024-05-05 16:26:54,835 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-05 16:26:54,954 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-05-05 16:26:54,954 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-05 16:26:55,078 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-05 16:26:55,079 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment
0,1,training,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",positive
1,2,training,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,positive
2,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive
3,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative
4,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive


Use desired model in generate sentiment

In [5]:
    model_id = 'google/flan-t5-xl'
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = T5ForConditionalGeneration.from_pretrained(model_id).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

  warnings.warn(
/data/homezvol1/anvieyra/.conda/envs/copilot/lib/python3.12/site-packages/huggingface_hub/file_download.py:1003: UserWarning: Not enough free disk space to download the file. The expected file size is: 9597.01 MB. The target location /data/homezvol1/anvieyra/.cache/huggingface/hub/models--google--flan-t5-xxl/blobs only has 9071.23 MB free disk space.
  warnings.warn(


OSError: [Errno 122] Disk quota exceeded

In [ ]:
def generate_t5xxl_sentiment(row: pd.Series) -> pd.Series:

    # extract dialogue from row and place into prompt.
    dialogue = row['dialogue_text']
    prompt = "Given the following dialogue, output a single digit representing the sentiment: " \
            "-1 for negative, 0 for neutral, and 1 for positive. Do not provide any additional " \
            " text or explanation.\n\n{}\n\nSentiment:".format(dialogue)

    # Encode the prompt to tensor
    input_ids = tokenizer.encode(prompt, return_tensors='pt', add_special_tokens=True).to(device)

    # Record GPU before processing
    gpu_before = GPUtil.getGPUs()[0]
    row['gpu_before'] = GPUtil.getGPUs()[0]
    row['load_before'] = gpu_before.load
    row['mem_before'] = gpu_before.memoryUsed
    row['start_time'] = time.time()

    # Generate output from the model
    generated_ids = model.generate(input_ids, max_new_tokens=100, do_sample=True)

    # Decode the generated ids to text
    decoded_output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Use regular expression to extract the sentiment value
    sentiment_match = re.search(r'Sentiment: (-?\d)', decoded_output)
    sentiment_value = int(sentiment_value.group(1)) if sentiment_match else 0
    row['generated_sentiment'] = sentiment_label[int(sentiment_value)]

    # Record GPU after processing
    row['gpu_after'] = GPUtil.getGPUs()[0]
    row['load_after'] = gpu_after.load
    row['mem_after'] = gpu_after.memoryUsed
    row['end_time'] = time.time()

    return row

In [ ]:

xxl_dialogue_data = generate_t5xxl_sentiment(dialogue_data)
xxl_dialogue_data.head()